In [3]:
import tensorflow as tf
import string
import requests
import pandas as pd

In [4]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')


In [5]:
data=response.text.splitlines()
print(data[:10]) 

['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough"]


In [6]:
len(data)

2400

In [7]:
# Total word number is:
len(" ".join(data))

91330

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data) #it's going to fit on the data in the forms of lines.

In [10]:
encoded_text=tokenizer.texts_to_sequences(data)
encoded_text[:10]

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [11]:
wc=tokenizer.word_counts
# word frequency

In [12]:
wi=tokenizer.word_index

In [13]:
print(f"Number of unique words and total vocab size: {len(tokenizer.word_counts)+1}")
vocab_size=len(tokenizer.word_counts)+1 # always adding plus one for tensorflow

Number of unique words and total vocab size: 1396


In [14]:
x=["play this song"]

In [15]:
tokenizer.texts_to_sequences(x)

[[241, 44, 409]]

In [16]:
 encoded_text[:10] # 10 sample

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [17]:
data_list=[]
for i in encoded_text:
    if len(i)>1:
        for j in range(2,len(i)):
            data_list.append(i[:j])
#             print(i[:j]) # if you want to check data

In [18]:
max_length=20
#max length of line is 20 token per line in our poetry
sequences=pad_sequences(data_list,maxlen=max_length,padding="pre") # we set the lenght size equal to 20
sequences[:5] # 5 sample

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 254,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 254,  21, 219],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 117,   8],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 117,   8,  80],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 117,   8,  80, 153]])

In [19]:
sequences.shape

(14231, 20)

In [20]:
X=sequences[:,:-1]
y=sequences[:,-1]
print("X values")
print(X[:5]) # 5 sample
print("-"*30)
print("X values")
print(y[:5]) # 5 sample

X values
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  254]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 254
   21]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  117]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117
    8]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117   8
   80]]
------------------------------
X values
[ 21 219   8  80 153]


In [21]:
X.shape,y.shape

((14231, 19), (14231,))

In [22]:
y=to_categorical(y,num_classes=vocab_size)
#since unique word number is vocab_size, thus there is vocab_size classes
print(y[:5]) # 5 sample
print("Shape of y",y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of y (14231, 1396)


In [23]:
print("Shape of X",X.shape)
seq_length=X.shape[1]
seq_length

Shape of X (14231, 19)


19

In [24]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length)) 
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) 

C:\Users\giree\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### You can use dropout() , You will see that Training will be slightly slower trend in convergence, maybe lower final accuracy

In [25]:
# model=Sequential()
# model.add(Embedding(vocab_size,50,input_length=seq_length)) 
# #The first layer is the Embedded layer that uses 50-length vectors
# #return_sequences=True because we add another LSTM
# model.add(Dropout(0.2))
# model.add(LSTM(100,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(100))
# model.add(Dropout(0.2))
# model.add(Dense(100,activation="relu"))
# model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

Dropout can be applied to the input and recurrent connections of the memory units with the LSTM precisely and separately.
LSTM-specific dropout may have more pronounced effect on the convergence of the network than the layer-wise dropout.

In [26]:
# model=Sequential()
# model.add(Embedding(vocab_size,50,input_length=seq_length)) 
# #The first layer is the Embedded layer that uses 50-length vectors
# #return_sequences=True because we add another LSTM
# model.add(LSTM(100,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
# model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
# model.add(Dense(100,activation="relu"))
# model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

In [27]:
# model=Sequential()
# model.add(Embedding(vocab_size,50,input_length=seq_length)) 
# #The first layer is the Embedded layer that uses 50-length vectors
# #return_sequences=True because we add another LSTM
# model.add(Conv1D(filters=32,kernel_size=3, padding="same",activation="relu"))
# model.add(MaxPooling1D(pool_size=2))
# model.add(LSTM(100,return_sequences=True))
# model.add(LSTM(100))
# model.add(Dense(100,activation="relu"))
# model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses 

In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [30]:
model.fit(X,y,batch_size=32,epochs=150)

Epoch 1/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.0365 - loss: 5.9939
Epoch 2/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.0501 - loss: 5.2611
Epoch 3/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.0588 - loss: 5.0875
Epoch 4/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.0622 - loss: 4.9086
Epoch 5/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.0801 - loss: 4.7703
Epoch 6/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.0921 - loss: 4.6464
Epoch 7/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.1034 - loss: 4.5340
Epoch 8/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.1124 - loss: 4.4348
Epoch 9/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.1364 - loss: 4.2513
Epoch 10/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.1553 - loss: 4.1127
Epoch 11/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.1739 - loss: 3.9421
Epoch 12/150
445/445 ━━━━

In [32]:
text_lenght= 15 # 15 words per line

def generate_text(input_text, no_lines):
    general_text=[]
    for i in range(no_lines):
        text=[]
        for _ in range(text_lenght):
            encoded=tokenizer.texts_to_sequences([input_text])
            encoded=pad_sequences(encoded,maxlen=seq_length,padding="pre")
            y_pred=np.argmax(model.predict(encoded),axis=-1) # it will generate a word index, loop up into dictionary containing word index
            
            predicted_word=""
            for word,index in tokenizer.word_index.items():
                if index==y_pred:
                    predicted_word=word
                    break
                    
            input_text=input_text +' '+ predicted_word
            text.append(predicted_word)
        
        input_text=text[-1]
        text=" ".join(text) # input text will be the last word of first created line
        general_text.append(text)
    
    return general_text

In [33]:
input_text="me"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━

["i just want to make love to you happy new year i heard that you're",
 'gonna wish you never had met me but you played it with a beating throw',
 'your stone through every open door as i had all you leave it for a',
 "thousand miles and poles hungry i will never be knocked down i can't keep up",
 'my heart you were smiling in the spotlight dancing with the way you leave it',
 'was just like a song i was so scared to face my love in the']

In [34]:
input_text="i want to see you"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━

["never felt an you want me to respect i can't go at trying me break",
 "how of your loving in the morning and pacing morning woulda again i'm thought don't",
 'let go baby give me light i miss you i miss you i remember all',
 'i ask is if this is my last night with you to love me why',
 "do you love me do you love me you said i'm stubborn and i never",
 'gonna leave you harder baby harder baby harder baby i can tell by the way']